Install or import the libaries and modules

In [ ]:
# Install libraries.
import numpy as np
from scipy import stats

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output

In [ ]:
# Import required libraries
#! pip install dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
from ipywidgets import interact, widgets

In [ ]:
import plotly.graph_objects as go

Get the data from Drive.

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define file paths
file_paths = {
    '2013': '/content/drive/MyDrive/Coding/Temporary Alaska Data/Slope 7 Scaled/2013_ranked.csv',
    '2015': '/content/drive/MyDrive/Coding/Temporary Alaska Data/Slope 7 Scaled/2015_ranked.csv',
    '2017': '/content/drive/MyDrive/Coding/Temporary Alaska Data/Slope 7 Scaled/2017_ranked.csv',
    '2018': '/content/drive/MyDrive/Coding/Temporary Alaska Data/Slope 7 Scaled/2018_ranked.csv',
    '2019': '/content/drive/MyDrive/Coding/Temporary Alaska Data/Slope 7 Scaled/2019_ranked.csv',
    '2020': '/content/drive/MyDrive/Coding/Temporary Alaska Data/Slope 7 Scaled/2020_ranked.csv'
}

# Load data into a dictionary of dataframes
dataframes = {year: pd.read_csv(path) for year, path in file_paths.items()}

# Show the first few rows of one of the dataframes to confirm it loaded correctly
print(dataframes['2013'].head())


Mounted at /content/drive
     Volume  Normalized Rank
0  0.896789         0.001524
1  0.772838         0.003049
2  0.645852         0.004573
3  0.336486         0.006098
4  0.313687         0.007622


Map the lengths of the observation periods into a dictionary for later reference.

In [ ]:
# Dictionary to map years to number of days
year_to_days = {
    '2013': 656,
    '2015': 812,
    '2017': 378,
    '2018': 347,
    '2019': 276,
    '2020': 366
}

# Verify by printing a sample
print("Sample year-to-days mapping for 2013:", year_to_days['2013'])


Sample year-to-days mapping for 2013: 656


Create a function to calculate the KS statistic.

In [ ]:
# Function to calculate the Kolmogorov-Smirnov statistic.
def MCF(year, vmin_limit):
    df = dataframes[year]  # Get the dataframe from the dictionary.
    days = year_to_days[year]  # Get the number of days in observation period.
    volumes = df['Volume']

    # Initialize an array to store vmin, D* pairs
    vmin_Dstar_pairs = []

    # Limit the value that volumes can take to only those below the top 1 order of magnitude.
    vmin_limit = min(vmin_limit, 10 ** (int(np.floor(np.log10(volumes.max()))) - 1))
#TODO: get rid of this vmin limit? why did i implement this again?
    # Truncate the data set
    truncated_volumes = volumes[volumes >= vmin_limit]
    n_truncated = len(truncated_volumes)

    # Calculate empirical CDF
    ecdf = np.arange(1, n_truncated + 1) / n_truncated

    # Calculate theoretical CDF
    log_values_theoretical = np.log(truncated_volumes / vmin_limit)
    b_hat = 1 + n_truncated * (np.sum(log_values_theoretical)) ** (-1)
    tcdf = 1 - (truncated_volumes / vmin_limit) ** (1 - b_hat)

    # Calculate D*, the weighted Kolmogorov-Smirnov statistic
    D_star, p_value = stats.ks_2samp(ecdf, tcdf)

    # Store current vmin, D*, and theoretical fit parameters
    vmin_Dstar_pairs.append((vmin_limit, D_star, p_value, b_hat, n_truncated))

    # Once done with the loop, store the vmin and D* where D* was minimized.
    vmin_best, D_star_best, p_value, b_hat_best, n_truncated_best = min(vmin_Dstar_pairs, key=lambda x: x[1])

    # Return the values where D* was minimized
    return vmin_best, D_star_best, p_value, b_hat_best, n_truncated_best


Create plotting functions, one for the MCF data with a power fit curve and one for viewing the empirical and theoretical CDFs.

In [ ]:
# TODO: define ecdf and tcdf function


# Use ipywidgets to create interactive plot

In [ ]:
# Create a dropdown for selecting the year
year_widget = widgets.Dropdown(
    options=['2013', '2015', '2017', '2018', '2019', '2020'],
    value='2013',
    description='Year:',
)

# Create a slider for selecting v_min, in a log scale
vmin_widget = widgets.FloatLogSlider(
    value=0.001,
    base=10,
    min=np.log10(0.0001), # max exponent of base
    max=np.log10(0.1), # min exponent of base
    step=0.1, # exponent step
    description='v_min:'
)

# Function to update plots
def update_plots(year, vmin):
    print(f"Selected Year: {year}, Selected vmin: {vmin}")
    # Here we will call the plotting and calculation functions
    # For now, it just prints the selected values

# Interactive widgets
interact(update_plots, year=year_widget, vmin=vmin_widget)


In [ ]:
def debug_update_plots(year, log_vmin):
    print(f"Debug - Selected Year: {year}, Selected vmin: {10 ** log_vmin}")  # Debug print

    # Try to print a sample from the selected dataframe
    print("Debug - Sample data from selected dataframe:")
    print(dataframes[year].head())

    vmin = 10 ** log_vmin  # Converting from log scale to linear scale

    # Run the MCF function and print its output
    vmin_best, D_star_best, p_value, b_hat_best, n_truncated_best = MCF(year)
    print(f"Debug - Output from MCF: {vmin_best, D_star_best, p_value, b_hat_best, n_truncated_best}")  # Debug print

    # Create a test plot with hardcoded values
    fig = go.Figure(data=go.Scatter(x=[1, 2, 3, 4], y=[1, 4, 9, 16], mode='markers'))
    fig.show()



In [ ]:
def update_plots(year, log_vmin):
    vmin = 10 ** log_vmin  # Converting from log scale to linear scale
    # Calculations
    vmin_best, D_star_best, p_value, b_hat_best, n_truncated_best = MCF(year, vmin)

    # Get the selected dataframe
    df = dataframes[year]

    # Create the plot for Normalized Rank vs. Volume
    fig = go.Figure(data=go.Scatter(x=df['Volume'], y=df['Normalized Rank'], mode='markers'))
    fig.add_trace(go.Scatter(x=[vmin_best, vmin_best], y=[df['Normalized Rank'].min(), df['Normalized Rank'].max()], mode='lines', name='Vmin Line'))

    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")

    fig.show()

    print(f"v_min: {vmin_best}, D*: {D_star_best}, p-value: {p_value}, b: {b_hat_best}, Number of events: {n_truncated_best}")

# Connect widgets to the function
#interact(update_plots, year=year_widget, log_vmin=vmin_widget)

# Connect widgets to the debug function
interact(debug_update_plots, year=year_widget, log_vmin=vmin_widget)


interactive(children=(Dropdown(description='Year:', options=('2013', '2015', '2017', '2018', '2019', '2020'), …

<function __main__.debug_update_plots(year, log_vmin)>

# Darn, come back to this later. Dash giving 403 error in colab.
(In the original layout, the Initialize Dash App cell came immediately before the "# Function to calculate the Kolmogorov-Smirnov statistic."

Create an interactive Dash app that will show the plots and parameters of interest.

In [ ]:
# Initialize the Dash App
app = dash.Dash(__name__)

# Define the Layout
app.layout = html.Div([
    dcc.Dropdown(
        id='year-selector',
        options=[{'label': year, 'value': year} for year in dataframes.keys()],
        value='2013',  # Default value
        style={'width': '50%'}
    ),
    dcc.Slider(
        id='vmin-slider',
        min=np.log10(0.0001),  # log10 of actual min
        max=np.log10(0.1),  # log10 of actual max
        step=0.1,  # Step in log10 scale
        value=np.log10(0.001),  # log10 of default value
        marks={np.log10(x): str(x) for x in [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1]}  # Logarithmic marks
    ),
    html.Div([
        dcc.Graph(id='data-plot'),
        dcc.Graph(id='cdf-plot')
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    html.Div(id='result-display')
])

# Define the Callback to Update the Plots
@app.callback(
    [Output('data-plot', 'figure'),
     Output('cdf-plot', 'figure'),
     Output('result-display', 'children')],
    [Input('year-selector', 'value'),
     Input('vmin-slider', 'value')]
)
def update_plots(selected_year, log_selected_vmin):
    # Convert log-selected vmin back to linear scale
    selected_vmin = 10 ** log_selected_vmin

    # Calculate values using the MCF function
    vmin_best, D_star_best, p_value, b_hat_best, n_truncated_best = MCF(selected_year, selected_vmin)

    # Get the selected dataframe
    df = dataframes[selected_year]

    # Create the plot for Normalized Rank vs. Volume
    fig1 = px.scatter(df, x='Volume', y='Normalized Rank', log_x=True, log_y=True)
    fig1.add_trace(go.Line(x=[vmin_best, vmin_best], y=[df['Normalized Rank'].min(), df['Normalized Rank'].max()], name='Vmin Line'))

    # Create the plot for Empirical and Theoretical CDF
    # TODO: right now it's just a dummy function for progression.
    # dummy plot function for development.
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=[1, 2, 3], y=[1, 4, 9], mode='lines', name='Dummy Line'))

    # Placeholder for the CDF plot
    #fig2 = go.Figure() #TODO: use this later when you replace the dummy function

    # Results to display
    result_text = f"v_min: {vmin_best}, D*: {D_star_best}, p-value: {p_value}, b: {b_hat_best}, Number of events: {n_truncated_best}"

    return fig1, fig2, result_text

# Run the app
if __name__ == '__main__':
  app.run_server(mode='external')


<IPython.core.display.Javascript object>